In [1]:
# Arguments
f='./depmap/Model.csv.gz'
host='localhost'
port='9999'
model='meta-llama/Meta-Llama-3-70B-Instruct'
#model='mistral-community/Mixtral-8x22B-v0.1'
# model = "guchia/Llama3-8B-Instruct_CanLit_LR_2-4_FT_0.1"
# model = "meta-llama/Meta-Llama-3-8B-Instruct"
# model = "guchia/Llama3-8B-Instruct_CanLit_LR_2-4_FT_0.01"
#model = "guchia/Llama3-8B-Instruct_CanLit_LR_2-4_FT_0.5"
#model = "guchia/Llama3-8B-Instruct_CanLit_LR_2-4_FT_1.0"
#model = "meta-llama/Meta-Llama-3-8B-Instruct"
model='llama31-405b-fp8'  # first time server was started with --served-model-name




In [4]:
parts = model.split("/")
# using --served-model-name messed up my code
if(len(parts) != 1):
    f_prefix = parts[1]
else:
    f_prefix = parts[0]
f_prefix = f_prefix + "_no_lora"
f_prefix

'part1_no_lora'

In [3]:
from openai import OpenAI

# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = 'cmsc-35360'
#openai_api_key = 'EMPTY'

openai_api_base = f"http://{host}:{port}/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [4]:
import pandas as pd

def load_data(f):
    df=pd.read_csv(f)
    print(f'done importing dataframe (rows, columns) = {df.shape}.')
    return df

In [5]:
import random

def select_random_element_not_equal_to(array, correct):
    '''
    Given an array of unique answers, select an answer not
    equal to the correct answer.
    '''
    if len(array) == 0:
        raise ValueError("The array is empty")

    # Filter out elements equal to `correct`
    filtered_array = [element for element in array if element != correct]
    
    if not filtered_array:
        raise ValueError("No valid elements to choose from")

    # Select a random element from the filtered list
    return random.choice(filtered_array)

In [6]:
def construct_question(df, index, array):
    '''
    Construct a multiple choice question from df using row at index.
    '''
    if index < 0 or index >= len(df):
        raise IndexError("Index out of bounds")
    
    cell_line_name = df.iloc[index]['CellLineName']
    correct_answer = df.iloc[index]['OncotreePrimaryDisease']

    wrong_answer_1 = select_random_element_not_equal_to(array, correct_answer)
    wrong_answer_2 = select_random_element_not_equal_to(array, correct_answer)
    wrong_answer_3 = select_random_element_not_equal_to(array, correct_answer)

    while wrong_answer_1 == wrong_answer_2 or wrong_answer_1 == wrong_answer_3 or wrong_answer_2 == wrong_answer_3:
        wrong_answer_1 = select_random_element_not_equal_to(array, correct_answer)
        wrong_answer_2 = select_random_element_not_equal_to(array, correct_answer)
        wrong_answer_3 = select_random_element_not_equal_to(array, correct_answer)
        
    answers = [correct_answer,
               wrong_answer_1,
               wrong_answer_2,
               wrong_answer_3]
    random.shuffle(answers)

    d = {"a": answers[0],
         "b": answers[1],
         "c": answers[2],
         "d": answers[3],
        }

    for choice in d.keys():
        if d[choice] == correct_answer:
            correct_choice = choice
    
    # question = f'''The tumor cell line named {cell_line_name} is
    # question = f'''The tumor cell line named {cell_line_name} is a biological model for which primary disease?
    question = f'''The cell line named {cell_line_name} is a biological model for which primary disease?

a) {d['a']}
b) {d['b']}
c) {d['c']}
d) {d['d']}'''
    
    return question, correct_choice, correct_answer, cell_line_name

In [7]:
import json
from tqdm import tqdm
from datetime import datetime

df = load_data(f)
array = df['OncotreePrimaryDisease'].unique()


sys_reg_prompt = '''You are a cancer research scientist studying the potential effects of various small molecules, peptides, and 
antibiodies on tumor cell growth. You will be presented with a series of multiple choice questions. Please select the correct
choice. Return the answer in json format {"CHOICE": "choice", "ANSWER": "answer"} where choice is only the alphabetic character 
associated with the full answer.'''

class MyException(Exception):
    pass

for i in range(0, 10):
    num_correct = 0
    total = 0
    responses = []
    response = {}


    for i in tqdm( range(df.shape[0]) ):
        # get question, correct choice and answer
        question, correct_choice, correct_answer, cell_line_name = construct_question(df, i, array)
        
        # construct message
        messages=[
            {"role": "system",
             "content": sys_reg_prompt
            },
            {"role": "user",
             "content": question
            },

        ]
        # print(f'{sys_reg_prompt}')
        # print(f'{question}')
        
        chat_response = client.chat.completions.create(
            model=model,
            # logprobs=1,
            # top_logprobs=1,
            messages=messages,
            #temperature=0.0,
            #max_tokens=2000,
            stream=False,
        )
        # print(f'{chat_response}')
        
        try:
            response = json.loads(chat_response.choices[0].message.content)
            response['CORRECT CHOICE'] = correct_choice
            response['CORRECT ANSWER'] = correct_answer
            response['CELL_LINE_NAME'] = cell_line_name
    
            if response['CHOICE'] == correct_choice:
                response['SCORE'] = 1
                num_correct = num_correct + 1
            else:
                response['SCORE'] = 0
    
            total = total + 1
            responses.append(response)

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

            response["CHOICE"] =  "e"
            response["ANSWER"] =  chat_response.choices[0].message.content
            response['CORRECT CHOICE'] = correct_choice
            response['CORRECT ANSWER'] = correct_answer
            response['CELL_LINE_NAME'] = cell_line_name
            response['SCORE'] = 0
            responses.append(response)

            print(f"QUESTION:\n{question}")
            print(f"MODEL RESPONSE:\n{chat_response.choices[0].message.content}")
            print(f"CORRECT RESPONSE:\n{correct_choice}. {correct_answer}")
            
            pass
    
    print(f'{num_correct} correct responses out of {total}')
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

    response_df = pd.DataFrame(responses)
    response_df.to_csv(f'{ofile_prefix}_model_name_eval_{timestamp}.tsv', index=None, sep="\t")

    with open(f'{ofile_prefix}_model_name_eval_summary.txt', 'a') as f:
        print(f'{timestamp}\t{num_correct} correct responses out of {total}', file=f)

done importing dataframe (rows, columns) = (1921, 36).


  3%|██                                                               | 60/1921 [01:01<33:53,  1.09s/it]

Error decoding JSON: Expecting ',' delimiter: line 1 column 72 (char 71)
QUESTION:
The cell line named HH is a biological model for which primary disease?

a) Squamous Cell Carcinoma of the Vulva/Vagina
b) Esophagogastric Adenocarcinoma
c) Non-Hodgkin Lymphoma
d) Ocular Melanoma
MODEL RESPONSE:
{"CHOICE": "a", "ANSWER": "Squamous Cell Carcinoma of the Vulva/Vagina"
CORRECT RESPONSE:
c. Non-Hodgkin Lymphoma


  5%|██▉                                                              | 88/1921 [01:28<30:15,  1.01it/s]

Error decoding JSON: Expecting ',' delimiter: line 1 column 52 (char 51)
QUESTION:
The cell line named NCI-H684 is a biological model for which primary disease?

a) Prostate Adenocarcinoma
b) Ovarian Germ Cell Tumor
c) Pancreatic Neuroendocrine Tumor
d) Colorectal Adenocarcinoma
MODEL RESPONSE:
{"CHOICE": "b", "ANSWER": "Ovarian Germ Cell Tumor"
CORRECT RESPONSE:
d. Colorectal Adenocarcinoma


...

 97%|████████████████████████████████████████████████████████████▊  | 1854/1921 [32:05<01:04,  1.04it/s]

Error decoding JSON: Expecting ',' delimiter: line 1 column 47 (char 46)
QUESTION:
The cell line named S462 is a biological model for which primary disease?

a) Pancreatic Neuroendocrine Tumor
b) Adrenocortical Carcinoma
c) Anaplastic Thyroid Cancer
d) Nerve Sheath Tumor
MODEL RESPONSE:
{"CHOICE": "d", "ANSWER": "Nerve Sheath Tumor"
CORRECT RESPONSE:
d. Nerve Sheath Tumor


100%|███████████████████████████████████████████████████████████████| 1921/1921 [33:14<00:00,  1.04s/it]


1483 correct responses out of 1905


NameError: name 'ofile_prefix' is not defined